In [3]:
# load in libraries
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score

Load in data

In [1]:
learned_data_path = '../processed_EEG_data\clean_learned_2025-02-23_14-24-48_1.csv'
not_learned_data_path = '../processed_EEG_data\clean_notLearned_2025-02-23_14-24-48_1.csv'

learned_data = pd.read_csv(learned_data_path)
not_learned_data = pd.read_csv(not_learned_data_path)

NameError: name 'pd' is not defined

Model


In [ ]:
# convert pd to a np array for the feature matrix X
X_learned = learned_data.values  
X_not_learned = not_learned_data.values

# make the label vector y
'''
to indicate which epochs go to which condition
y vector label should be as long as x matrix is long

zeros = learned
ones = not learned
'''
y_learned = np.zeros(X_learned.shape[0])
y_not_learned = np.ones(X_not_learned.shape[0])

# combine the data and labels
# learned + not learned 
# learned labels + not learned labels
X = np.concatenate([X_learned, X_not_learned], axis=0)
y = np.concatenate([y_learned, y_not_learned], axis=0)

# split into training and test data
test_size_n = 0.5   # sets this percentage aside for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_n, random_state=0)

model = SVC(kernel='rbf') #from SVM scikit-learn library, uses radial basis function

# train the data using fit
model.fit(X_train, y_train)

# make predictions based on the testing data from before
y_pred = model.predict(X_test)

# evauluate the performance of the SVM model by caclulating confusino matrix
# and the accuracy score
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix: \n", cm)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

Confusion Matrix: 
 [[27  0]
 [ 2  0]]
Accuracy:  0.9310344827586207
